In [13]:
%serialconnect --port=COM3 --baudrate=115200

serial exception on close WriteFile failed (PermissionError(13, 'Access is denied.', None, 5))
Connecting to --port=COM3 --baud=115200 
16.39344
Traceback (most recent call last):
  File "boot.py", line 17, in <module>
KeyboardInterrupt: 
MicroPython v0.5.0-46-g78dafab5a-dirty on 2020-06-20; Sipeed_M1 with kendryte-k210
Type "help()" for more information.
>>> 
>>> 
>>> 
raw REPL; CTRL-B to exit
>
Ready.


In [14]:
from machine import I2C
import time

In [15]:
i2c = I2C(I2C.I2C0, freq=100000, scl=35, sda=34)

In [16]:
print(i2c.scan ())

[18]


In [6]:
i2c.writeto(0x12,bytes([int(128),int(50),int(0)]))

In [7]:
i2c.writeto(0x12,bytes([int(128),int(200),int(0)]))

In [9]:
i2c.writeto(0x12,bytes([int(240),int(128),int(0)]))

In [10]:
i2c.writeto(0x12,bytes([int(28),int(128),int(0)]))

In [11]:
import struct
tong = struct.pack(">H", 1)
print(tong)
i2c.writeto(0x12,tong)

b'\x00\x01'


In [17]:
data = i2c.readfrom(0x12, 3)
print(data)

b'\x80\x80\x00'


In [18]:
while 1 :
    data = i2c.readfrom(0x12, 3)
    print(data)
    time.sleep(1)

b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x00'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\xbc\x04'
b'\x80\x1c\x04'
b'\x80\xa6\x04'
b'\x80\x80\x04'
b'\x80\xe4\x04'
b'\x80\x80\x04'
b'\xc8\x80\x04'
b'/\x80\x04'
b'\x1c\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b',\x80\x04'
b')\x80\x04'
b'\x1c\x80\x04'
b'\x1c\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'
b'\x80\x80\x04'


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
KeyboardInterrupt: 


In [20]:
import uos

In [21]:
print(uos.listdir("/sd"))

['System Volume Information', 'main.py']


In [ ]:
tong = uos.listdir("/sd")

In [ ]:
print(tong[1])

In [ ]:
# Untitled - By: ER9X - Fri Nov 8 2019

import sensor, image, time,lcd
from Maix import freq


print(freq.get())

#freq.set(500,500)

lcd.init()
lcd.clear()
sensor.reset(freq=24000000,dual_buff=True)
sensor.set_pixformat(sensor.RGB565)
sensor.set_framesize(sensor.QVGA)
sensor.set_vflip(1)
sensor.run(1)
#sensor.set_windowing((0,0,224, 224))
#sensor.run(1)
lcd.init(type=2,freq=20000000,color=(255,0,0))
#time.sleep(1)
#lcd.init(type=2, freq=20000000, color=(0,255,0))
#time.sleep(1)
#lcd.init(type=2, freq=20000000, color=(0,0,255))
#time.sleep(1)
clock = time.clock()

while(True):
    clock.tick()
    img = sensor.snapshot()
    #tong = img.rgb_to_lab()
    #img2 = img.rgb_to_grayscale()
    lcd.display(img)
    fps =clock.fps()
    print(fps)

In [ ]:
import sensor, image, time ,lcd
import KPU as kpu
from fpioa_manager import fm
from machine import I2C
from board import board_info
from Maix import GPIO
i2c = I2C(I2C.I2C0, freq=100000, scl=35, sda=34)

sensor.reset(freq=35000000,dual_buff=True)
sensor.set_pixformat(sensor.RGB565)
sensor.set_framesize(sensor.QVGA)
#sensor.skip_frames(time = 2000)
sensor.set_vflip(1)
sensor.run(1)

clock = time.clock()
toss1 = 0


lcd.init()

task = kpu.load(0x300000)

#task = kpu.load("/sd/t6.kmodel")
##datazzz = bytes([int(128),int(128),int(0)])
a = kpu.set_outputs(task,0,1,1,1)
try:
    while(True):
        toss = toss1
        toss1 = time.ticks_ms()
        print(1000/(toss1-toss))
        #img = image.Image("/sd/dataset/55.jpg")
        img = sensor.snapshot()
        #img = img.draw_rectangle(0,0,320,80,color=(0,0,0),fill=True)
        ##img = img.draw_rectangle(130,180,90,240-180,color=(0,0,0),fill=True)
        img = img.resize(160,160)

        ##img = img.draw_rectangle(0,180,224,320,color=(0,0,0),fill=True)
        a = img.pix_to_ai()
        #print(img[0])

        fmap = kpu.forward(task,img)
        plist=fmap[:]
        #pmax=max(plist)
        #max_index=plist.index(pmax)
        #print(plist)
        txx = 0
        if abs(plist[0]) >0.2 :
            txx = (plist[0])*128

        #print(txx)
        ##txx = txx*1.5
        if txx > 127 :
            txx = 127
        if txx < -127 :
            txx = -127
        #print(txx)
        #datazzz = bytes([int(128),int(txx+127),int(0)])
        ####print(datazzz)
        #i2c.writeto(0x12,bytes([int(200),int(txx+127),int(0)]))
        #lcd.display(img)
        #lcd.draw_string(90, 0, str(int(1000/(toss1-toss)))+" fps", lcd.RED,lcd.BLACK)
        #lcd.display(img2)

except KeyboardInterrupt :
    a = kpu.deinit(task)